# Debug notebook
This notebook is only used to troubleshoot the S1 ARD workflow processor.

## S1 ARD workflow for scenes overlapping AM

In [1]:
import timeout_decorator
import json
from utils.prepS1AM import prepareS1AM

@timeout_decorator.timeout(10800, use_signals=False)
def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    logging.info("%s: %s" % (x, loaded_json[x]))
    prepareS1AM(**loaded_json)

## S1 ARD job queue processor

In [2]:
import os
import logging
import rediswq

level = os.getenv("LOGLEVEL", "INFO").upper()
logging.basicConfig(format="%(asctime)s %(levelname)-8s %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=level)

host = os.getenv("REDIS_SERVICE_HOST", "redis-master")
q = rediswq.RedisWQ(name="jobS1AM", host=host)

logging.info("Worker with sessionID: " +  q.sessionID())
logging.info("Initial queue state: empty=" + str(q.empty()))

2020-10-17 11:12:51 INFO     Worker with sessionID: 7810206b-c1b4-4404-b823-e6af438a5f0c
2020-10-17 11:12:51 INFO     Initial queue state: empty=False


In [3]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        logging.info("Working on " + itemstr)

        # In case COG conversion gets stuck, a TimeoutError is raised and we try again: the existance of a COG from an earlier iteration is enough to progress upon retrying
        for x in range(0, 2):  # try 2 times
            e = False
            try:
                process_scene(itemstr)
                e = True
            except timeout_decorator.TimeoutError:
                logging.info("Timed out while working on " + itemstr)
                e = False
                pass
            if e:
                break

        q.complete(item)
    else:
        logging.info("Waiting for work")

2020-10-17 11:12:54 INFO     Working on {"in_scene": "S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6", "s3_bucket": "public-eo-data", "s3_dir": "luigi/fiji/sentinel_1/"}
2020-10-17 11:12:54 INFO     S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6.SAFE S1A_IW_GRDH_1SSV_20161215T173221 Starting
2020-10-17 11:12:54 INFO     S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6.SAFE S1A_IW_GRDH_1SSV_20161215T173221 DOWNLOADING via ASF
2020-10-17 11:12:54 INFO     fetching: https://api.daac.asf.alaska.edu/services/search/param?granule_list=S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6&output=csv
2020-10-17 11:12:55 INFO     logging into asf
2020-10-17 11:12:58 INFO     successfully logged into asf
2020-10-17 11:13:31 INFO     S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6.SAFE S1A_IW_GRDH_1SSV_20161215T173221 DOWNLOADED via ASF
2020-10-17 11:13:31 INFO     S1A_IW_GRDH_1SSV_20161215T173221_20161

not boo


In [4]:
print("Queue empty!")

Queue empty!


---